<a href="https://colab.research.google.com/github/weasel-codes/covid-patient-recovery/blob/main/Covid19_Patient_Recovery_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relevant imports

In [1]:
import numpy as np
import matplotlib.pyplot as plot
import pandas as pd

# Importing Dataset

In [2]:
dataset = pd.read_csv('PatientInfo.csv')
# print("\n\nAll Columns : ")
# print(dataset.info())
dataset = dataset[{'sex','age','infection_case','confirmed_date','released_date','deceased_date', 'state'}]
# print("\n\nRelevant Columns : ")
# print(dataset.info())
# print("Shape of Dataset : ", dataset.shape)
print("No. of Nan values per column : ")
dataset.isnull().sum()

No. of Nan values per column : 


age               1380
deceased_date     5099
sex               1122
confirmed_date       3
released_date     3578
state                0
infection_case     919
dtype: int64

In [3]:
print("Shape of dataset before removing Nan from any column : ", dataset.shape)

dataset = dataset.dropna(subset=['confirmed_date'], how='all')
# print("\nShape of dataset after removing Nan from confirmed_date : ", dataset.shape)
dataset = dataset[dataset.state != 'isolated']
# print("\nShape of dataset after removing Nan from column state = isolated : ", dataset.shape)
# print("No. of Nan values per column after removing Nan values from column state = isolated : ")
# print(dataset.isnull().sum())
dataset = dataset.dropna(subset=['deceased_date', 'released_date'], how='all')
# print("\nShape of dataset after removing Nan from column deceased and released date : ", dataset.shape)
# print("No. of Nan values per column after removing released, deceased, confirmed date : ")
# print(dataset.isnull().sum())
dataset['sex'].fillna(method="ffill", inplace = True)
dataset['age'].fillna(method="ffill", inplace = True)
dataset['infection_case'].fillna(method="ffill", inplace = True)
print("No. of Nan values per column after removing Nans : ")
print(dataset.isnull().sum())

Shape of dataset before removing Nan from any column :  (5165, 7)
No. of Nan values per column after removing Nans : 
age                  0
deceased_date     1580
sex                  0
confirmed_date       0
released_date       64
state                0
infection_case       0
dtype: int64


# Generate Array out of Dataset

In [4]:
dataset=dataset.iloc[:].values
dataset[:10]

array([['50s', nan, 'male', '2020-01-23', '2020-02-05', 'released',
        'overseas inflow'],
       ['30s', nan, 'male', '2020-01-30', '2020-03-02', 'released',
        'overseas inflow'],
       ['50s', nan, 'male', '2020-01-30', '2020-02-19', 'released',
        'contact with patient'],
       ['20s', nan, 'male', '2020-01-30', '2020-02-15', 'released',
        'overseas inflow'],
       ['20s', nan, 'female', '2020-01-31', '2020-02-24', 'released',
        'contact with patient'],
       ['50s', nan, 'female', '2020-01-31', '2020-02-19', 'released',
        'contact with patient'],
       ['20s', nan, 'male', '2020-01-31', '2020-02-10', 'released',
        'contact with patient'],
       ['20s', nan, 'male', '2020-02-02', '2020-02-24', 'released',
        'overseas inflow'],
       ['30s', nan, 'male', '2020-02-05', '2020-02-21', 'released',
        'overseas inflow'],
       ['60s', nan, 'female', '2020-02-05', '2020-02-29', 'released',
        'contact with patient']], dtype=ob

In [5]:
for i in range(len(dataset)):
  dataset[i,0] = pd.to_numeric(dataset[i,0].replace('s',''))
  print(dataset[i])


[50 nan 'male' '2020-01-23' '2020-02-05' 'released' 'overseas inflow']
[30 nan 'male' '2020-01-30' '2020-03-02' 'released' 'overseas inflow']
[50 nan 'male' '2020-01-30' '2020-02-19' 'released' 'contact with patient']
[20 nan 'male' '2020-01-30' '2020-02-15' 'released' 'overseas inflow']
[20 nan 'female' '2020-01-31' '2020-02-24' 'released'
 'contact with patient']
[50 nan 'female' '2020-01-31' '2020-02-19' 'released'
 'contact with patient']
[20 nan 'male' '2020-01-31' '2020-02-10' 'released' 'contact with patient']
[20 nan 'male' '2020-02-02' '2020-02-24' 'released' 'overseas inflow']
[30 nan 'male' '2020-02-05' '2020-02-21' 'released' 'overseas inflow']
[60 nan 'female' '2020-02-05' '2020-02-29' 'released'
 'contact with patient']
[50 nan 'female' '2020-02-06' '2020-02-29' 'released' 'overseas inflow']
[20 nan 'male' '2020-02-07' '2020-02-27' 'released' 'overseas inflow']
[60 nan 'female' '2020-02-16' '2020-03-12' 'released'
 'contact with patient']
[70 nan 'male' '2020-02-19' '2020